In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten, concatenate, Input, Dropout, LSTM, GRU, Bidirectional,BatchNormalization,PReLU,ReLU,Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt


2022-07-10 12:17:01.197476: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [14]:

init_df = pd.read_csv('./csv/sample_size_523_output_gameemo.csv',  sep=',')

print('Shape of data: ', init_df.shape)

Shape of data:  (58576, 2703)


In [15]:

df = init_df.copy()
print(df.head())

#HA_PV = high arousal, positive valence
#HA_NV = high arousal, negative valence
#LA_NV = low arousal, negative valence
#LA_PV = low arousal, positive valance

label_map = {1:"HA_PV", 2:"HA_NV", 3:"LA_NV", 4:"LA_PV"}

df["Label"] = df["Label"].map(label_map)


   lag1_mean_0  lag1_mean_1  lag1_mean_2  lag1_mean_3  lag1_mean_4  \
0    24.879771     3.106697    -1.239877    -0.555734   -17.584463   
1    -0.440098    -1.449679     9.304872     1.034350    -0.725376   
2     0.829206    -3.484418    -1.650308     0.111657     0.766096   
3     1.930660    -1.260540     8.699489     2.875833     7.494725   
4     3.259741   -20.054091    -2.742413     2.732820   -23.647910   

   lag1_mean_5  lag1_mean_6  lag1_mean_7  lag1_mean_8  lag1_mean_9  ...  \
0   -17.821559   -12.738755   -16.915080     2.374827    -2.715987  ...   
1    -2.137903    -0.015417     0.565977    -0.992917     1.060846  ...   
2    -4.615465    -1.194480    -0.970423     0.958364     4.277605  ...   
3    -3.528349     3.550838     0.847434    -0.780957    -1.208350  ...   
4    -7.475699    -4.147064     0.274457     5.449123     6.032328  ...   

   freq_282_13  freq_292_13  freq_302_13  freq_313_13  freq_323_13  \
0     0.015183     0.007817     0.016656     0.011823     

In [16]:

print(df.head())

features = df.iloc[:, :-1]
label = df.iloc[:, -1:]

print('Shape of data: ', df.shape)
print('features.shape: ', features.shape)
print('label.shape: ', label.shape)

# df.head()
# print(df.columns)

y = label.to_numpy()
X = features.to_numpy()

############################################################
# total number of batches: 8064 
# timesteps per batch: 72
# features: 8974 (minus one for the label at the end)

num_of_features = X.shape[1]

# sample_size = 72  
# num_of_features = 8974

sample_size = 523  

train_dataset_percentage = 0.7

print("sample_size:",sample_size)
print("num_of_features:",num_of_features)

total_samples_count = int(X.shape[0]/sample_size)

print("total_samples_count:", total_samples_count)


train_sample_count = int(total_samples_count * train_dataset_percentage)
test_sample_count = total_samples_count - train_sample_count

train_size = train_sample_count * sample_size
test_size = test_sample_count * sample_size

print("train size:", train_size)
print("test size:", test_size)

X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print("train.shape:", X_train.shape)
print("test.shape:", X_test.shape)

X_train = X_train.reshape((train_sample_count,sample_size,num_of_features))
X_test = X_test.reshape((test_sample_count,sample_size,num_of_features))

print("X_train.shape after reshape:",X_train.shape)
print("X_test.shape after reshape:",X_test.shape)

#collapse y_train and y_test to the same X sample counts instead

y_train_collapsed = np.array([])
for i in range(len(y_train)):
    if (i % sample_size == 0):
        y_train_collapsed = np.append(y_train_collapsed, (y_train[i]))
        
print("y_train_collapsed shape:",y_train_collapsed.shape)        

y_test_collapsed = np.array([])
for i in range(len(y_test)):
    if (i % sample_size == 0):
        y_test_collapsed = np.append(y_test_collapsed, (y_test[i]))
        
print("y_test_collapsed shape:",y_test_collapsed.shape)    

# one-hot encoding
y_train = pd.get_dummies(y_train_collapsed)
y_test = pd.get_dummies(y_test_collapsed)

print("y_train.shape:", y_train.shape)
print("y_test.shape:", y_test.shape)



   lag1_mean_0  lag1_mean_1  lag1_mean_2  lag1_mean_3  lag1_mean_4  \
0    24.879771     3.106697    -1.239877    -0.555734   -17.584463   
1    -0.440098    -1.449679     9.304872     1.034350    -0.725376   
2     0.829206    -3.484418    -1.650308     0.111657     0.766096   
3     1.930660    -1.260540     8.699489     2.875833     7.494725   
4     3.259741   -20.054091    -2.742413     2.732820   -23.647910   

   lag1_mean_5  lag1_mean_6  lag1_mean_7  lag1_mean_8  lag1_mean_9  ...  \
0   -17.821559   -12.738755   -16.915080     2.374827    -2.715987  ...   
1    -2.137903    -0.015417     0.565977    -0.992917     1.060846  ...   
2    -4.615465    -1.194480    -0.970423     0.958364     4.277605  ...   
3    -3.528349     3.550838     0.847434    -0.780957    -1.208350  ...   
4    -7.475699    -4.147064     0.274457     5.449123     6.032328  ...   

   freq_282_13  freq_292_13  freq_302_13  freq_313_13  freq_323_13  \
0     0.015183     0.007817     0.016656     0.011823     

In [17]:
def train_model(x_train, y_train,x_test,y_test, save_to, epoch, sample_size, num_of_features):
    strategy = tf.distribute.MirroredStrategy(devices=None)
    print('Number of devices: {}'.format(strategy.num_replicas_in_sync))  
    
    tf.keras.backend.clear_session()
    tf.random.set_seed(0)
    
    with strategy.scope():

#         inputs = tf.keras.Input(shape=(sample_size,num_of_features))
#         #ml_model = tf.keras.layers.GRU(256, return_sequences=True)(inputs)
#         ml_model = tf.keras.layers.LSTM(256, return_sequences=True)(inputs)
#         flat = Flatten()(ml_model)
#         outputs = Dense(4, activation='softmax')(flat)
#         model = tf.keras.Model(inputs, outputs)
        
        ######
        #sample size:38252/524, accuracy: 1.0000 - val_loss: 5.5889 - val_accuracy: 0.4403
        #sample size:38252/73, loss: 1.4733e-04 - accuracy: 1.0000 - val_loss: 4.6835 - val_accuracy: 0.5475
        model = Sequential()
        model.add(LSTM(256, return_sequences=True, input_shape=(sample_size,num_of_features)))

        model.add(Flatten())
        model.add(Dense(4))
        model.add(Activation('softmax'))        
        
        ######
#         model = Sequential()
#         model.add(LSTM(256, return_sequences=True, input_shape=(sample_size,num_of_features), go_backwards=True))
#         model.add(Flatten())
#         model.add(Dense(4))
#         model.add(Activation('softmax'))

        ######
        
#         model = Sequential()
#         model.add(Bidirectional(LSTM(256, return_sequences=True), 
#                                 input_shape=(sample_size,num_of_features))) #, merge_mode='concat'))
#         model.add(Flatten())
#         model.add(Dense(4))
#         model.add(Activation('softmax')) 
        

        model.summary()
        tf.keras.utils.plot_model(model)

        opt_adam = keras.optimizers.Adam(learning_rate=0.001)

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
        mc = ModelCheckpoint(save_to + '_best_model_lstm_time_domain.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
            
        lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001 * np.exp(-epoch / 10.))
            
        model.compile(optimizer=opt_adam,
                      loss=['categorical_crossentropy'],
                      metrics=['accuracy'])

          
    history = model.fit(x_train,y_train,
                        batch_size=32,
                        epochs=epoch,
                        validation_data=(x_test,y_test),
                        callbacks=[es,mc,lr_schedule], shuffle=False)
        
    # saved_model = load_model(save_to + '_best_model_lstm_all_cat.h5')
        
    return model,history



In [18]:

model,history = train_model(X_train, y_train,X_test, y_test, save_to= './', epoch = 40, 
                            sample_size=sample_size, num_of_features=num_of_features)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
Number of devices: 1
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 523, 256)          3030016   
_________________________________________________________________
flatten (Flatten)            (None, 133888)            0         
_________________________________________________________________
dense (Dense)                (None, 4)                 535556    
_________________________________________________________________
activation (Activation)      (None, 4)                 0         
Total params: 3,565,572
Trainable params: 3,565,572
Non-trainable params: 0
_________________________________________________________________


2022-07-10 12:45:34.325811: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_6166"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.aut

Epoch 1/40
3/3 [==============================] - ETA: 0s - loss: 4.2561 - accuracy: 0.2962

2022-07-10 12:45:40.331348: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:656] In AUTO-mode, and switching to DATA-based sharding, instead of FILE-based sharding as we cannot find appropriate reader dataset op(s) to shard. Error: Did not find a shardable source, walked to a node which is not a dataset: name: "FlatMapDataset/_9"
op: "FlatMapDataset"
input: "PrefetchDataset/_8"
attr {
  key: "Targuments"
  value {
    list {
    }
  }
}
attr {
  key: "f"
  value {
    func {
      name: "__inference_Dataset_flat_map_slice_batch_indices_8777"
    }
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: -1
        }
      }
    }
  }
}
attr {
  key: "output_types"
  value {
    list {
      type: DT_INT64
    }
  }
}
. Consider either turning off auto-sharding or switching the auto_shard_policy to DATA to shard this dataset. You can do this by creating a new `tf.data.Options()` object then setting `options.experimental_distribute.aut

3/3 [==============================] - 8s 2s/step - loss: 4.7401 - accuracy: 0.2959 - val_loss: 11.0415 - val_accuracy: 0.2353

Epoch 00001: val_accuracy improved from -inf to 0.23529, saving model to ./_best_model_lstm_time_domain.h5
Epoch 2/40
3/3 [==============================] - 3s 1s/step - loss: 6.8877 - accuracy: 0.2091 - val_loss: 5.1344 - val_accuracy: 0.2059

Epoch 00002: val_accuracy did not improve from 0.23529
Epoch 3/40
3/3 [==============================] - 3s 1s/step - loss: 0.6832 - accuracy: 0.7493 - val_loss: 4.0848 - val_accuracy: 0.2059

Epoch 00003: val_accuracy did not improve from 0.23529
Epoch 4/40
3/3 [==============================] - 3s 1s/step - loss: 0.1526 - accuracy: 0.9431 - val_loss: 2.0326 - val_accuracy: 0.2353

Epoch 00004: val_accuracy did not improve from 0.23529
Epoch 5/40
3/3 [==============================] - 3s 1s/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 2.1505 - val_accuracy: 0.3235

Epoch 00005: val_accuracy improved from 0.23529 

In [ ]:
def plot_loss_history(history):
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.xticks(range(0, len(history['loss'] + 1)))
  plt.plot(history['loss'], label="training", marker='o')
  plt.plot(history['val_loss'], label="validation", marker='o')
  plt.legend()
  plt.show()

def plot_accuracy_history(history):
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.xticks(range(0, len(history['accuracy'] + 1)))
  plt.plot(history['accuracy'], label="training", marker='o')
  plt.plot(history['val_accuracy'], label="validation", marker='o')
  plt.legend()
  plt.show()

In [ ]:

history_data = pd.DataFrame(history.history)
plot_loss_history(history_data)
plot_accuracy_history(history_data)